In [2]:
from rdkit import Chem
from rdkit.Chem.SaltRemover import SaltRemover
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import Descriptors
import numpy as np
import pandas as pd
import networkx as nx
import torch
from rdkit.Chem.rdchem import BondType
from calculate_delta_energy.gap_deltaenergy import *
import pickle
from sklearn.feature_selection import VarianceThreshold
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [3]:
your_smis = ['CC(C(=O)c1ccc(O)cc1O)c1ccc(O)cc1', 'COC1=CC=C(\C=C\C2=CC(OC)=C(O)C(OC)=C2)C=C1']
your_mols = [Chem.MolFromSmiles(smi) for smi in your_smis]
your_nbegapenergy, your_smiles_list =GetNBEandGapEnergy_asfeatures(your_mols, your_smis)

df_your_smi = pd.DataFrame({'standard_smiles':your_smiles_list})
df_your_gapenergy = pd.DataFrame(your_nbegapenergy, columns= col_list)
df_your_gapenergy = pd.concat([df_your_smi, df_your_gapenergy],axis=1)

df_your_gapenergy_fe = FE(df_your_gapenergy)
print(df_your_gapenergy_fe.shape)
df_your_all = descri(df_your_gapenergy_fe)

(2, 226)


In [4]:
df_your_gapenergy_fe

,standard_smiles,sx_ave1,sx_ave2,sx_ave3,sx_ave4,sx_ave5,sx_ave6,sx_ave7,sx_ave8,sx_ave9,...,nx_std6*nx_std7,nx_std6*nx_std8,nx_std6*nx_std9,nx_std6*nx_std10,nx_std7*nx_std8,nx_std7*nx_std9,nx_std7*nx_std10,nx_std8*nx_std9,nx_std8*nx_std10,nx_std9*nx_std10
0,CC(C(=O)c1ccc(O)cc1O)c1ccc(O)cc1,146.796296,97.164557,132.604167,111.036585,105.87500,103.387097,102.784314,96.342857,98.117647,...,9308.715216,10175.613565,10296.350685,10397.066227,10837.632205,10966.224402,11073.492430,11987.482610,12104.740254,12248.367119
1,COC1=CC=C(\C=C\C2=CC(OC)=C(O)C(OC)=C2)C=C1,126.250000,83.779221,120.290000,86.968750,96.62069,88.000000,80.718310,104.888889,84.314815,...,8849.215605,8796.595781,7955.333653,8410.429280,9309.724331,8419.389172,8901.031723,8369.325211,8848.103786,8001.915691


In [5]:
df_your_all

,standard_smiles,sx_ave1,sx_ave2,sx_ave3,sx_ave4,sx_ave5,sx_ave6,sx_ave7,sx_ave8,sx_ave9,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,CC(C(=O)c1ccc(O)cc1O)c1ccc(O)cc1,146.796296,97.164557,132.604167,111.036585,105.87500,103.387097,102.784314,96.342857,98.117647,...,0,0,0,0,0,0,0,0,0,0
1,COC1=CC=C(\C=C\C2=CC(OC)=C(O)C(OC)=C2)C=C1,126.250000,83.779221,120.290000,86.968750,96.62069,88.000000,80.718310,104.888889,84.314815,...,0,0,0,0,0,0,0,0,0,0


In [6]:
gapen_all = ['sx_ave1', 'sx_ave2', 'sx_ave3', 'sx_ave4', 'sx_ave5', 'sx_ave6', 'sx_ave7', 'sx_ave8', 'sx_ave9', 'sx_ave10', 'sx_std1', 'sx_std2', 'sx_std3', 'sx_std4', 'sx_std5', 'sx_std6', 'sx_std7', 'sx_std8', 'sx_std9', 'sx_std10', 'nx_ave1', 'nx_ave2', 'nx_ave3', 'nx_ave4', 'nx_ave5', 'nx_ave6', 'nx_ave7', 'nx_ave8', 'nx_ave9', 'nx_ave10', 'nx_std1', 'nx_std2', 'nx_std3', 'nx_std4', 'nx_std5', 'nx_std6', 'nx_std7', 'nx_std8', 'nx_std9', 'nx_std10', 'oldnbe', 'oldstd', 'newnbe', 'newstd', 'max_gap', 'sx_ave1*sx_ave2', 'sx_ave1*sx_ave3', 'sx_ave1*sx_ave4', 'sx_ave1*sx_ave5', 'sx_ave1*sx_ave6', 'sx_ave1*sx_ave7', 'sx_ave1*sx_ave8', 'sx_ave1*sx_ave9', 'sx_ave1*sx_ave10', 'sx_ave2*sx_ave3', 'sx_ave2*sx_ave4', 'sx_ave2*sx_ave5', 'sx_ave2*sx_ave6', 'sx_ave2*sx_ave7', 'sx_ave2*sx_ave8', 'sx_ave2*sx_ave9', 'sx_ave2*sx_ave10', 'sx_ave3*sx_ave4', 'sx_ave3*sx_ave5', 'sx_ave3*sx_ave6', 'sx_ave3*sx_ave7', 'sx_ave3*sx_ave8', 'sx_ave3*sx_ave9', 'sx_ave3*sx_ave10', 'sx_ave4*sx_ave5', 'sx_ave4*sx_ave6', 'sx_ave4*sx_ave7', 'sx_ave4*sx_ave8', 'sx_ave4*sx_ave9', 'sx_ave4*sx_ave10', 'sx_ave5*sx_ave6', 'sx_ave5*sx_ave7', 'sx_ave5*sx_ave8', 'sx_ave5*sx_ave9', 'sx_ave5*sx_ave10', 'sx_ave6*sx_ave7', 'sx_ave6*sx_ave8', 'sx_ave6*sx_ave9', 'sx_ave6*sx_ave10', 'sx_ave7*sx_ave8', 'sx_ave7*sx_ave9', 'sx_ave7*sx_ave10', 'sx_ave8*sx_ave9', 'sx_ave8*sx_ave10', 'sx_ave9*sx_ave10', 'sx_std1*sx_std2', 'sx_std1*sx_std3', 'sx_std1*sx_std4', 'sx_std1*sx_std5', 'sx_std1*sx_std6', 'sx_std1*sx_std7', 'sx_std1*sx_std8', 'sx_std1*sx_std9', 'sx_std1*sx_std10', 'sx_std2*sx_std3', 'sx_std2*sx_std4', 'sx_std2*sx_std5', 'sx_std2*sx_std6', 'sx_std2*sx_std7', 'sx_std2*sx_std8', 'sx_std2*sx_std9', 'sx_std2*sx_std10', 'sx_std3*sx_std4', 'sx_std3*sx_std5', 'sx_std3*sx_std6', 'sx_std3*sx_std7', 'sx_std3*sx_std8', 'sx_std3*sx_std9', 'sx_std3*sx_std10', 'sx_std4*sx_std5', 'sx_std4*sx_std6', 'sx_std4*sx_std7', 'sx_std4*sx_std8', 'sx_std4*sx_std9', 'sx_std4*sx_std10', 'sx_std5*sx_std6', 'sx_std5*sx_std7', 'sx_std5*sx_std8', 'sx_std5*sx_std9', 'sx_std5*sx_std10', 'sx_std6*sx_std7', 'sx_std6*sx_std8', 'sx_std6*sx_std9', 'sx_std6*sx_std10', 'sx_std7*sx_std8', 'sx_std7*sx_std9', 'sx_std7*sx_std10', 'sx_std8*sx_std9', 'sx_std8*sx_std10', 'sx_std9*sx_std10', 'nx_ave1*nx_ave2', 'nx_ave1*nx_ave3', 'nx_ave1*nx_ave4', 'nx_ave1*nx_ave5', 'nx_ave1*nx_ave6', 'nx_ave1*nx_ave7', 'nx_ave1*nx_ave8', 'nx_ave1*nx_ave9', 'nx_ave1*nx_ave10', 'nx_ave2*nx_ave3', 'nx_ave2*nx_ave4', 'nx_ave2*nx_ave5', 'nx_ave2*nx_ave6', 'nx_ave2*nx_ave7', 'nx_ave2*nx_ave8', 'nx_ave2*nx_ave9', 'nx_ave2*nx_ave10', 'nx_ave3*nx_ave4', 'nx_ave3*nx_ave5', 'nx_ave3*nx_ave6', 'nx_ave3*nx_ave7', 'nx_ave3*nx_ave8', 'nx_ave3*nx_ave9', 'nx_ave3*nx_ave10', 'nx_ave4*nx_ave5', 'nx_ave4*nx_ave6', 'nx_ave4*nx_ave7', 'nx_ave4*nx_ave8', 'nx_ave4*nx_ave9', 'nx_ave4*nx_ave10', 'nx_ave5*nx_ave6', 'nx_ave5*nx_ave7', 'nx_ave5*nx_ave8', 'nx_ave5*nx_ave9', 'nx_ave5*nx_ave10', 'nx_ave6*nx_ave7', 'nx_ave6*nx_ave8', 'nx_ave6*nx_ave9', 'nx_ave6*nx_ave10', 'nx_ave7*nx_ave8', 'nx_ave7*nx_ave9', 'nx_ave7*nx_ave10', 'nx_ave8*nx_ave9', 'nx_ave8*nx_ave10', 'nx_ave9*nx_ave10', 'nx_std1*nx_std2', 'nx_std1*nx_std3', 'nx_std1*nx_std4', 'nx_std1*nx_std5', 'nx_std1*nx_std6', 'nx_std1*nx_std7', 'nx_std1*nx_std8', 'nx_std1*nx_std9', 'nx_std1*nx_std10', 'nx_std2*nx_std3', 'nx_std2*nx_std4', 'nx_std2*nx_std5', 'nx_std2*nx_std6', 'nx_std2*nx_std7', 'nx_std2*nx_std8', 'nx_std2*nx_std9', 'nx_std2*nx_std10', 'nx_std3*nx_std4', 'nx_std3*nx_std5', 'nx_std3*nx_std6', 'nx_std3*nx_std7', 'nx_std3*nx_std8', 'nx_std3*nx_std9', 'nx_std3*nx_std10', 'nx_std4*nx_std5', 'nx_std4*nx_std6', 'nx_std4*nx_std7', 'nx_std4*nx_std8', 'nx_std4*nx_std9', 'nx_std4*nx_std10', 'nx_std5*nx_std6', 'nx_std5*nx_std7', 'nx_std5*nx_std8', 'nx_std5*nx_std9', 'nx_std5*nx_std10', 'nx_std6*nx_std7', 'nx_std6*nx_std8', 'nx_std6*nx_std9', 'nx_std6*nx_std10', 'nx_std7*nx_std8', 'nx_std7*nx_std9', 'nx_std7*nx_std10', 'nx_std8*nx_std9', 'nx_std8*nx_std10', 'nx_std9*nx_std10', ]
descri = ['MaxEStateIndex', 'MinEStateIndex', 'MaxAbsEStateIndex', 'MinAbsEStateIndex', 'qed', 'MolWt', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons', 'NumRadicalElectrons', 'MaxPartialCharge', 'MinPartialCharge', 'MaxAbsPartialCharge', 'MinAbsPartialCharge', 'FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3', 'BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW', 'BalabanJ', 'BertzCT', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v', 'HallKierAlpha', 'Ipc', 'Kappa1', 'Kappa2', 'Kappa3', 'LabuteASA', 'PEOE_VSA1', 'PEOE_VSA10', 'PEOE_VSA11', 'PEOE_VSA12', 'PEOE_VSA13', 'PEOE_VSA14', 'PEOE_VSA2', 'PEOE_VSA3', 'PEOE_VSA4', 'PEOE_VSA5', 'PEOE_VSA6', 'PEOE_VSA7', 'PEOE_VSA8', 'PEOE_VSA9', 'SMR_VSA1', 'SMR_VSA10', 'SMR_VSA2', 'SMR_VSA3', 'SMR_VSA4', 'SMR_VSA5', 'SMR_VSA6', 'SMR_VSA7', 'SMR_VSA8', 'SMR_VSA9', 'SlogP_VSA1', 'SlogP_VSA10', 'SlogP_VSA11', 'SlogP_VSA12', 'SlogP_VSA2', 'SlogP_VSA3', 'SlogP_VSA4', 'SlogP_VSA5', 'SlogP_VSA6', 'SlogP_VSA7', 'SlogP_VSA8', 'SlogP_VSA9', 'TPSA', 'EState_VSA1', 'EState_VSA10', 'EState_VSA11', 'EState_VSA2', 'EState_VSA3', 'EState_VSA4', 'EState_VSA5', 'EState_VSA6', 'EState_VSA7', 'EState_VSA8', 'EState_VSA9', 'VSA_EState1', 'VSA_EState10', 'VSA_EState2', 'VSA_EState3', 'VSA_EState4', 'VSA_EState5', 'VSA_EState6', 'VSA_EState7', 'VSA_EState8', 'VSA_EState9', 'FractionCSP3', 'HeavyAtomCount', 'NHOHCount', 'NOCount', 'NumAliphaticCarbocycles', 'NumAliphaticHeterocycles', 'NumAliphaticRings', 'NumAromaticCarbocycles', 'NumAromaticHeterocycles', 'NumAromaticRings', 'NumHAcceptors', 'NumHDonors', 'NumHeteroatoms', 'NumRotatableBonds', 'NumSaturatedCarbocycles', 'NumSaturatedHeterocycles', 'NumSaturatedRings', 'RingCount', 'MolLogP', 'MolMR', 'fr_Al_COO', 'fr_Al_OH', 'fr_Al_OH_noTert', 'fr_ArN', 'fr_Ar_COO', 'fr_Ar_N', 'fr_Ar_NH', 'fr_Ar_OH', 'fr_COO', 'fr_COO2', 'fr_C_O', 'fr_C_O_noCOO', 'fr_C_S', 'fr_HOCCN', 'fr_Imine', 'fr_NH0', 'fr_NH1', 'fr_NH2', 'fr_N_O', 'fr_Ndealkylation1', 'fr_Ndealkylation2', 'fr_Nhpyrrole', 'fr_SH', 'fr_aldehyde', 'fr_alkyl_carbamate', 'fr_alkyl_halide', 'fr_allylic_oxid', 'fr_amide', 'fr_amidine', 'fr_aniline', 'fr_aryl_methyl', 'fr_azide', 'fr_azo', 'fr_barbitur', 'fr_benzene', 'fr_benzodiazepine', 'fr_bicyclic', 'fr_diazo', 'fr_dihydropyridine', 'fr_epoxide', 'fr_ester', 'fr_ether', 'fr_furan', 'fr_guanido', 'fr_halogen', 'fr_hdrzine', 'fr_hdrzone', 'fr_imidazole', 'fr_imide', 'fr_isocyan', 'fr_isothiocyan', 'fr_ketone', 'fr_ketone_Topliss', 'fr_lactam', 'fr_lactone', 'fr_methoxy', 'fr_morpholine', 'fr_nitrile', 'fr_nitro', 'fr_nitro_arom', 'fr_nitro_arom_nonortho', 'fr_nitroso', 'fr_oxazole', 'fr_oxime', 'fr_para_hydroxylation', 'fr_phenol', 'fr_phenol_noOrthoHbond', 'fr_phos_acid', 'fr_phos_ester', 'fr_piperdine', 'fr_piperzine', 'fr_priamide', 'fr_prisulfonamd', 'fr_pyridine', 'fr_quatN', 'fr_sulfide', 'fr_sulfonamd', 'fr_sulfone', 'fr_term_acetylene', 'fr_tetrazole', 'fr_thiazole', 'fr_thiocyan', 'fr_thiophene', 'fr_unbrch_alkane', 'fr_urea']

In [7]:
xgb_gapen_important_features = np.load("data/xgb_gapen_important_features.npy", allow_pickle=True)
xgb_descri_important_features = np.load("data/xgb_descri_important_features.npy", allow_pickle=True)

In [8]:
# df_exo = pd.read_csv('data/exo_gapen_descri_0106_sxnx.csv', index_col=0)
# df_tox = pd.read_csv('data/tox_gapen_descri_0106_sxnx.csv', index_col=0)
# df_all = pd.concat([df_exo, df_tox], axis=0)
# df_all_random = shuffle(df_all,  random_state=1)

# def get_mean_std_feature(df=df_all_random):
#     features = df.iloc[:,2:]
#     labels = df['label']
#     selector = VarianceThreshold(0.0) #排除为0的特征
#     selector = selector.fit(features)
#     features = selector.transform(features)  # 会变成数组
#     features = pd.DataFrame(features)
#     X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=1,
#                                                         stratify=labels)
#     print(X_train.shape)
#     df_mean = np.array(X_train.mean(skipna=True))
#     df_std = np.array(X_train.std(skipna=True) + 0.00001)
#     return  df_mean, df_std, selector

# df_mean, df_std, selector = get_mean_std_feature()

# feature_names = selector.get_feature_names_out(selector.feature_names_in_)
# np.save("gap-delta-energy/data/feature_names.npy", feature_names)
# np.save("gap-delta-energy/data/feature_df_mean.npy", df_mean)
# np.save("gap-delta-energy/data/feature_df_std.npy", df_std)


In [9]:
feature_names = np.load("gap-delta-energy/data/feature_names.npy", allow_pickle=True)
df_mean= np.load("gap-delta-energy/data/feature_df_mean.npy")
df_std= np.load("gap-delta-energy/data/feature_df_std.npy")

def get_feature(df_your_all = df_your_all):
    features = df_your_all.iloc[:,1:]
    features = features[feature_names]
    
    # features = np.array(features)
    print(features.shape)
    # df_mean = 
    # df_std = 
    features = (features - df_mean) / df_std
    features = features.fillna(0.0)
    return features

your_features = get_feature()

(2, 429)


In [10]:
your_features

,sx_ave1,sx_ave2,sx_ave3,sx_ave4,sx_ave5,sx_ave6,sx_ave7,sx_ave8,sx_ave9,sx_ave10,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,1.130753,0.691290,1.163965,0.714169,0.525316,0.547412,0.616812,0.560271,0.631787,0.028803,...,-0.191016,-0.068355,-0.048275,-0.054313,-0.011301,-0.101031,-0.028617,-0.067561,-0.407879,-0.074021
1,0.649340,0.198216,0.816990,-0.000309,0.264859,0.112719,0.011059,0.794839,0.249002,-0.243460,...,-0.191016,-0.068355,-0.048275,-0.054313,-0.011301,-0.101031,-0.028617,-0.067561,-0.407879,-0.074021


In [11]:
# load XGB model with selected features
def pickle_XGBmodel_test(clf_str,  features = your_features):
    with open('models/'+clf_str+'.pkl', 'rb') as f:
        clf = pickle.load(f)
    if clf_str == 'xgb_gap':
        X_test =  features[xgb_gapen_important_features]
    elif clf_str == 'xgb_des':
        X_test =  features[xgb_descri_important_features]
    y_test_pred = clf.predict(X_test)
    y_test_probas = clf.predict_proba(X_test)[:, 1]
    print(f"{clf_str}: {y_test_probas}")

    return y_test_probas

xgb_gapen_test_proba = pickle_XGBmodel_test('xgb_gap')
xgb_descri_test_proba = pickle_XGBmodel_test('xgb_des')

xgb_gap: [0.9152476  0.00117221]
xgb_des: [0.66470337 0.00414977]


In [12]:
# load the LR model
def pickle_LRmodel_test(X_test_proba):

    with open('models/'+'XGB_LR'+'.pkl', 'rb') as f:
        clf = pickle.load(f)
    y_test_pred = clf.predict(X_test_proba)
    y_test_probas = clf.predict_proba(X_test_proba)[:, 1]
    print("XGB-LR-stacking" ,y_test_probas)
    return y_test_probas

X_test_proba = np.concatenate([xgb_gapen_test_proba[:,np.newaxis], xgb_descri_test_proba[:,np.newaxis]], axis=1)
y_test_probas = pickle_LRmodel_test(X_test_proba)

XGB-LR-stacking [0.88751931 0.0773445 ]
